<a href="https://colab.research.google.com/github/adnanhilal/Hilal_CNG562_HW1/blob/master/Hilal_forest_fire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error as mse
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [4]:
url='http://www.dsi.uminho.pt/~pcortez/forestfires/forestfires.csv'
fire=pd.read_csv(url)
fire.columns

Index(['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH',
       'wind', 'rain', 'area'],
      dtype='object')

## data conversion using encoding

In [0]:
categorical_features = ['month',
                       'day']

categorical_transformer = Pipeline(steps=[('onehot',
OneHotEncoder(handle_unknown='ignore'))])

ct = ColumnTransformer(transformers=[('categorical_transformer',
                                      categorical_transformer,
                                      categorical_features)],
remainder='passthrough')
datain = ct.fit_transform(fire.drop('area', axis=1))
dataout = fire['area']

## data separation

In [0]:
x_train, x_test, y_train, y_test = train_test_split(datain,dataout,test_size=0.3, random_state=30)

# original data without scaling

## knn model 

In [0]:
knnreg = KNeighborsRegressor(n_neighbors=30, p=3)
knn = knnreg.fit(x_train, y_train)
print('mse for train knn:', mse (y_train, knn.predict(x_train)))
print('mse for test knn:', mse(y_test, knn.predict(x_test)))

mse for train knn: 5471.498419220682
mse for test knn: 870.0147768931623


## linear regression model

In [0]:
reg = LinearRegression().fit(x_train, y_train)
print('mse for train linearregression:', mse (y_train, reg.predict(x_train)))
print('mse for train linearregression:', mse (y_test, reg.predict(x_test)))

mse for train linearregression: 5251.76524109501
mse for train linearregression: 830.8707221992546


# scaled data analysis

In [0]:
X_scaled = preprocessing.scale(datain)
Y_scaled = preprocessing.scale(dataout)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled,y_scaled,test_size=0.30, random_state=17)

## knn model

In [0]:
knnreg = KNeighborsRegressor(n_neighbors=12, p=3)
knn = knnreg.fit(x_train, y_train)
print('mse for train knn:', mse (y_train, knn.predict(x_train)))
print('mse for test knn:', mse (y_test, knn.predict(x_test)))

mse for train knn: 0.9656493643120783
mse for test knn: 1.0937239599680293


## leanear model

In [0]:
reg = LinearRegression().fit(x_train, y_train)
print('mse for train linearregression:', mse (y_train, reg.predict(x_train)))
print('mse for test linearregression:', mse (y_test, reg.predict(x_test)))

mse for train linearregression: 0.9474844516671346
mse for test linearregression: 15.33252069034272


# adding features

In [0]:
#datain

In [0]:
rain_square = fire.rain * fire.rain
firerain=np.c_[ datain, rain_square]
temp_square = fire.temp * fire.temp
fire2=np.c_[ firerain, temp_square]
wind_square = fire.wind * fire.wind
fire3=np.c_[ fire2, wind_square]
wind_temp = fire.wind * fire.temp
fire4=np.c_[ fire3, wind_temp]
#fire4

In [0]:
x_scaled = preprocessing.scale(fire4)
y_scaled = preprocessing.scale(dataout)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled,y_scaled,test_size=0.30, random_state=17)

## knn model

### knn training

In [0]:
knnreg = KNeighborsRegressor(n_neighbors=25, p=12)
knn = knnreg.fit(x_train, y_train)
print('mse for train knn:', mse (y_train, knn.predict(x_train)))
print('mse for test knn:', mse (y_test, knn.predict(x_test)))

mse for train knn: 0.9973240499485118
mse for test knn: 1.0243419838164947


### knn grid search

In [0]:
knnreg = KNeighborsRegressor()
parameters = {'n_neighbors': (3,5,10,20,30,35,45,80), 'p': (2,3,5,10,12,40)}
gscvknn = GridSearchCV(estimator = knnreg, param_grid = parameters, n_jobs=-1,cv=5 )
gscvknn.fit(x_train, y_train)
print("best_params_knn" ,(gscvknn.best_params_)) 
print("accuracy_score_knn_train" ,(gscvknn.score(x_test, y_test)))
print ("accuracy_score_knn_test" ,(gscvknn.score(x_train, y_train)))

C:\Users\adnan\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


best_params_knn {'n_neighbors': 80, 'p': 40}
accuracy_score_knn_train -0.02607776886735702
accuracy_score_knn_test 0.018801236830076618


## decision tree model

### decision tree trainig

In [0]:
from sklearn.tree import DecisionTreeRegressor
tree= DecisionTreeRegressor(max_depth=5, max_leaf_nodes=3, random_state=150).fit(x_train, y_train)
print('mse for train tree:', mse (y_train, tree.predict(x_train)))
print('mse for test tree:', mse (y_test, tree.predict(x_test)))

mse for train tree: 0.2194813868754099
mse for test tree: 0.9480065073055439


### decision tree grid search

In [0]:
tree= DecisionTreeRegressor()
parameters = {'max_depth': (0.1,0.2,1,3,5,10), 'max_leaf_nodes': (2,3,5,10,12,40)}
gscvtree = GridSearchCV(estimator = tree, param_grid = parameters, n_jobs=-1,
cv=5, )
gscvtree.fit(x_train, y_train)
print("best_params_tree" ,(gscvtree.best_params_)) 
print("accuracy_score_tree_train" ,(gscvtree.score(x_test, y_test)))
print ("accuracy_score_tree_test" ,(gscvtree.score(x_train, y_train)))

best_params_tree {'max_depth': 0.1, 'max_leaf_nodes': 2}
accuracy_score_tree_train -0.10773718924403841
accuracy_score_tree_test 0.04497354497354511


## lianear model 

In [0]:
reg = LinearRegression().fit(x_train, y_train)
print('mse for train reg:', mse (y_train, reg.predict(x_train)))
print('mse for test reg:', mse (y_test, reg.predict(x_test)))

mse for train reg: 0.9474844516671346
mse for test reg: 15.33252069034272


# converting fire area to binary

In [0]:
dataoutb= 1.0 * (fire['area']>40) ## threshold=40
#dataout

In [0]:
x_trainb, x_testb, y_trainb, y_testb = train_test_split(x_scaled,dataoutb,test_size=0.3, random_state=4)

## knn model, training and grid search (binary classification)

In [0]:
knnreg = KNeighborsClassifier(n_neighbors=40, p=3)
knn = knnreg.fit(x_train, y_train)
print("accuracy_score_knnb_train" ,(knn.score(x_trainb, y_trainb)))
print ("accuracy_score_knnb_test" ,(knn.score(x_testb, y_testb)))

accuracy_score_knnb_train 0.9418282548476454
accuracy_score_knnb_test 0.9358974358974359


In [0]:
from sklearn.model_selection import GridSearchCV
knnreg = KNeighborsClassifier()
parameters = {'n_neighbors': (3,5,10,20,30,35,45,80), 'p': (2,3,5,10,12,40)}
gscvknnb = GridSearchCV(estimator = knnreg, param_grid = parameters, n_jobs=-1,cv=5 )
gscvknnb.fit(x_train, y_train)
print("best_params_knnb" ,(gscvknnb.best_params_)) 
print("accuracy_score_knnb_train" ,(gscvknnb.score(x_test, y_test)))
print ("accuracy_score_knnb_test" ,(gscvknnb.score(x_train, y_train)))

best_params_knnb {'n_neighbors': 3, 'p': 2}
accuracy_score_knnb_train 0.9358974358974359
accuracy_score_knnb_test 0.9473684210526315


## tree model, training and grid search (binary classification)

In [0]:
treeb = DecisionTreeClassifier(max_leaf_nodes=5).fit(x_trainb, y_trainb)
print("accuracy_score_treeb_train" ,(treeb.score(x_trainb, y_trainb)))
print ("accuracy_score_treeb_test" ,(treeb.score(x_testb, y_testb)))

accuracy_score_treeb_train 0.9501385041551247
accuracy_score_treeb_test 0.9102564102564102


In [0]:
treeb= DecisionTreeClassifier()
parameters = {'max_depth': (5,10,15,80,78), 'max_leaf_nodes': (2,4,7,9,20,40)}
gscvtreeb = GridSearchCV(estimator = treeb, param_grid = parameters, n_jobs=-1,
cv=5, )
gscvtreeb.fit(x_trainb, y_trainb)

print("best_params_treeb" ,(gscvtreeb.best_params_)) 
print("accuracy_score_treeb_train" ,(gscvtreeb.score(x_testb, y_testb)))
print ("accuracy_score_treeb_test" ,(gscvtreeb.score(x_trainb, y_trainb)))

best_params_treeb {'max_depth': 5, 'max_leaf_nodes': 2}
accuracy_score_treeb_train 0.9294871794871795
accuracy_score_treeb_test 0.9445983379501385


## knn model, training  (binary classification)

In [0]:
regb = LogisticRegression().fit(x_trainb, y_trainb)
print("accuracy_score_regb_train" ,(regb.score(x_testb, y_testb)))
print ("accuracy_score_regb_test" ,(regb.score(x_trainb, y_trainb)))

accuracy_score_regb_train 0.9294871794871795
accuracy_score_regb_test 0.9418282548476454


none of the models was able to represent the data with a good accuracy, (they weren't a good conversion) . the limited data is the reason of the high error. adding more features wasn't efficant to make a notacable increase of the acurecy. on the other hand,  convert the problem to binary increase and enhance the accurecy score.